<a href="https://colab.research.google.com/github/MShahabSepehri/CryptoMamba/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Cloning
!git clone https://github.com/MShahabSepehri/CryptoMamba.git
%cd CryptoMamba/

fatal: destination path 'CryptoMamba' already exists and is not an empty directory.
/content/CryptoMamba


In [2]:
# @title Installing requirements

%%capture
!pip install mamba-ssm[causal-conv1d] --no-build-isolation
!pip install -r requirements.txt

In [3]:
# @title Prediction
!python scripts/one_day_pred.py --config cmamba_v --ckpt_path ./checkpoints/cmamba_v.ckpt --date 2024-12-15

['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']

Prediction date: 2024-12-15
Prediction: 102174.26
Today value: 101372.97
Smart trade: 39.21% buy
Vanilla trade: -


In [ ]:
# @title Training
!python scripts/training.py --config cmamba_v